# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
%load_ext dotenv
%dotenv ../../01_materials/labs/.env


In [2]:
import dask
#dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

/tmp/ipykernel_83552/3123200313.py:3: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

  import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

parquet_files = glob(os.path.join(os.getenv('PRICE_DATA'), '*/*.parquet/*.parquet'))
parquet_files

['../../05_src/data/prices/ODFL/ODFL_2002.parquet/part.0.parquet',
 '../../05_src/data/prices/ODFL/ODFL_2006.parquet/part.0.parquet',
 '../../05_src/data/prices/ODFL/ODFL_2021.parquet/part.0.parquet',
 '../../05_src/data/prices/ODFL/ODFL_2017.parquet/part.0.parquet',
 '../../05_src/data/prices/ODFL/ODFL_2013.parquet/part.0.parquet',
 '../../05_src/data/prices/ODFL/ODFL_2016.parquet/part.0.parquet',
 '../../05_src/data/prices/ODFL/ODFL_2012.parquet/part.0.parquet',
 '../../05_src/data/prices/ODFL/ODFL_2003.parquet/part.0.parquet',
 '../../05_src/data/prices/ODFL/ODFL_2007.parquet/part.0.parquet',
 '../../05_src/data/prices/ODFL/ODFL_2024.parquet/part.0.parquet',
 '../../05_src/data/prices/ODFL/ODFL_2020.parquet/part.0.parquet',
 '../../05_src/data/prices/ODFL/ODFL_2014.parquet/part.0.parquet',
 '../../05_src/data/prices/ODFL/ODFL_2010.parquet/part.0.parquet',
 '../../05_src/data/prices/ODFL/ODFL_2018.parquet/part.0.parquet',
 '../../05_src/data/prices/ODFL/ODFL_2022.parquet/part.0.parqu

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# import numpy as np
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Adj_Close_lag = x['Adj Close'].shift(1))
).assign(
    returns = lambda x: x['Adj Close']/x['Adj_Close_lag'] - 1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))

dd_feat

/home/kostiantyn/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/dask/dataframe/core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(
/tmp/ipykernel_83552/1863668572.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Adj_Close_lag,returns,hi_lo_range
npartitions=11217,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,string,string,int32,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [5]:
dd_feat.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Adj_Close_lag,returns,hi_lo_range
ticker,,,,,,,,,,,,,
IP,2017-01-03,50.293560,50.994316,50.151516,50.558712,36.379150,2034912,Materials,Paper & Plastic Packaging Products & Materials,2017,NaN,NaN,0.842800
IP,2017-01-04,50.795456,51.297348,50.662880,51.183712,36.828865,2461536,Materials,Paper & Plastic Packaging Products & Materials,2017,36.379150,0.012362,0.634468
IP,2017-01-05,51.041668,51.382576,50.501896,50.767044,36.529060,2197642,Materials,Paper & Plastic Packaging Products & Materials,2017,36.828865,-0.008140,0.880680
IP,2017-01-06,50.719696,51.070076,50.435604,50.871212,36.604004,1323696,Materials,Paper & Plastic Packaging Products & Materials,2017,36.529060,0.002052,0.634472
IP,2017-01-09,50.852272,51.297348,50.587120,50.776516,36.535873,3154272,Materials,Paper & Plastic Packaging Products & Materials,2017,36.604004,-0.001861,0.710228
...,...,...,...,...,...,...,...,...,...,...,...,...,...
BXP,2020-12-24,93.199997,94.010002,91.470001,93.930000,77.627068,367300,Real Estate,Office REITs,2020,76.527924,0.014363,2.540001
BXP,2020-12-28,93.959999,95.480003,93.379997,95.139999,78.627060,762400,Real Estate,Office REITs,2020,77.627068,0.012882,2.100006
BXP,2020-12-29,95.580002,96.449997,93.010002,93.290001,77.098152,820500,Real Estate,Office REITs,2020,78.627060,-0.019445,3.439995


In [6]:
dd_feat['Avg_Rolling_Return'] = dd_feat['returns'].rolling(10).mean()

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Was it necessary to convert to pandas to calculate the moving average return? - No, I performed calculation before converting, and it worked
Would it have been better to do it in Dask? Why? - I assume yes, as calculations are made faster in Dask and files with same content have lower size

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.